In [1]:
# Import create_engine, MetaData
from sqlalchemy import create_engine, MetaData

# Define an engine to connect to chapter5.sqlite: engine
engine = create_engine('sqlite:///K:\TensorflowPY36CPU\TensorflowPY36CPU\_1_PythonBasic\Database\chapter5.sqlite')

# Initialize MetaData: metadata
metadata = MetaData()


In [2]:
# Import Table, Column, String, and Integer
from sqlalchemy import (Table, Column, String, Integer)

census = Table('census', metadata,
               Column('state', String(30)),
               Column('sex', String(1)),
               Column('age', Integer()),
               Column('pop2000', Integer()),
               Column('pop2008', Integer()))

# Create the table in the database

metadata.create_all(engine)

# Reading the Data from the CSV

In [3]:
# Create an empty list: values_list
values_list = []

# Iterate over the rows
for row in csv_reader:
    # Create a dictionary with the values
    data = {'state': row[0], 'sex': row[1], 'age': row[2], 'pop2000': row[3],
            'pop2008': row[4]}
    # Append the dictionary to the values list
    values_list.append(data)

NameError: name 'csv_reader' is not defined